# Bayesian Modeling of Registration-time Fake Hotel Detection for Uncertainty Quantification

## Bayesian modeling of registration-time fake hotel detection
The idea here is to make a model that predicts whether a newly registrated property is fake while quantifying the **epistemic uncertainty** of the prediction. In other words: we want the model **to know when it doesn't know**. 

This is achieved by applying Variational Bayes Logistic Regression, where each of the weights of the model are themselves a represented by a Normal distribution instead of being just a scalar number. The mean and variance of the Normal distribution of each normal distribution is learned from the data. This results in a situation where, if there is low uncertainty about the model weights, the variances will shrink to zero. At prediction time, the model will generate a distribution of model scores for each instance rather than a single prediction. This means that we can generate multiple predictions for the same instance by sampling from the weight distributions and applying these sampled weights to make a prediction. When a prediction is made that was in a dense-part of the feature space in the training set, we expect the model to generate narrow prediction intervals. When a prediction is made for an instance that was unlike any instance in the training set, we expect the model to generate wide prediction intervals, representing the uncertainty of the generated prediction.

This is hypothesized to have several possible use cases:

- if the model's uncertainty grows over time, then this can possibly be used as a measure of the degree to which the model is impacted by concept drift
- the model's uncertainty could then be used as a measure for when the model needs to be retrained
- the uncertain predictions of the model could be send to fraud operations team for manual investigation as a form of active learning

This notebook takes several steps:

- it trains a [lightGBM](https://lightgbm.readthedocs.io/en/latest/) model to learn the complex feature-interactions that we need in fraud detection
- we use this lightGBM model to transform each instance of the Batman data into so-called **leaf-node assignment**, i.e., for each instance on which it predicts it creates a one-hot-encoded vector for each tree of the lightGBM tree-ensemble that indicates to which leaf-node of this tree this instance belongs. The one-hot-encoded vectors of the individual trees are concatenated to create a vector of zero-and-ones as a learned feature representation of the instance based on the whole tree ensemble.
- it fits a Variational Bayes logistic regression on top of the leaf-node assignments.
- we measure the uncertainty of the predictions, and show that instances that signed up a long time after the model had been trained had more uncertainty than instances that signed up just after the model had been trained.

In [ ]:
from pyspark.sql import functions as sf

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop

import tensorflow as tf
import tensorflow_probability as tfp
import os
import numpy as np
import matplotlib.pyplot as plt
import lightgbm
import seaborn as sns
import pandas as pd

tfd = tfp.distributions
tfpl = tfp.layers

### Just some data plumbing & pre-processing

In [ ]:
batman_pd = (
    spark.table("counterfraud.batman_model_features_and_labels")
    .where(sf.col('signup_finished_at')>='2019-06-01')
    .toPandas()
)

In [ ]:
# sort data by signup date to create a stream that would resemble reality
# use mergesort as it is a stable sort (unlike the default quicksort)
batman_pd.sort_values("signup_finished_at", kind="mergesort", inplace=True)

In [ ]:
# all the feature columns in the Batman dataset are prefixed with "property_"
feature_cols = [col for col in batman_pd if col.startswith('property__')]

X = batman_pd[feature_cols]
y = batman_pd["is_fake_hotel"].to_numpy()

In [ ]:
def preprocess_training_data(X):
    """
    Pre-processes feature vectors by:
    - filling None and NaN values with -1
    - label_encoding categorical variables

    Our dataset contains no features that naturally take take the value -1,
    so this will effectively allow the tree to seperate our None/NaNs from
    other values

    In our normal model training pipeline, H2O takes care of null values
    and handling of categorical variables. With lightgbm we need
    to do that ourselves.
    """
    categorical_feature_indices = [i for i, x in enumerate(X.dtypes) if x == "object"]
    if categorical_feature_indices:
        preprocess_categorical_features(X, categorical_feature_indices)
    # preprocess NaN values in numeric features, as skmultiflow's learners can't handle them
    X.fillna(-1, inplace=True)
    return X


def preprocess_categorical_features(df, indices):
    # fill None or NaN values
    df.iloc[:, indices] = df.iloc[:, indices].fillna("")
    # LabelEncode categorical features
    df.iloc[:, indices] = df.iloc[:, indices].apply(LabelEncoder().fit_transform)

In [ ]:
batman_pd[feature_cols] = preprocess_training_data(X)

In [ ]:
batman_pd

In [ ]:
def batman_data_daterange(date_start, date_end):
    # Just a helper function to get the Batman instances of a given date range
    return batman_pd[(batman_pd["signup_finished_at"] >= date_start) 
                     & (batman_pd["signup_finished_at"] < date_end)]

In [ ]:
def to_lightgbm_dataset(data):
    # Just a helper function to get some Batman instances in the format that LightGBM likes
    return lightgbm.Dataset(data[feature_cols], label=data["is_fake_hotel"], categorical_feature=[])

### Training the LightGBM model

In [ ]:
parameters = {
    # parameters as found in: https://vizier.booking.com/dashboard/studies/2428
    'application': 'binary',
    'objective': 'binary',
    'metric': 'auc',
    'boosting': 'gbdt',
    "max_depth": 13,
    "learning_rate": 0.024782155042265455,
    "reg_alpha": 0.2927510698213333,
    "reg_lambda": 0.2509836997410686,
    "colsample_bytree": 0.6481804503911828,
    "min_child_weight": 67,
    "num_iterations": 25,
}

model = lightgbm.train(parameters,
                       to_lightgbm_dataset(batman_data_daterange("2020-04-01", "2020-08-01")),
                       valid_sets=to_lightgbm_dataset(batman_data_daterange("2020-08-01", "2020-09-01")),
                       num_boost_round=5000,
                       early_stopping_rounds=100)

In [ ]:
leaf_node_assignments = model.predict(
    batman_data_daterange("2020-04-01", "2020-09-01")[feature_cols],
    pred_leaf=True,
)

In [ ]:
oh_enc = OneHotEncoder(
    handle_unknown='ignore',
    sparse=False
)

In [ ]:
oh_enc.fit(leaf_node_assignments)

In [ ]:
oh_leaf_nodes = oh_enc.transform(leaf_node_assignments)

### Building the Variational Bayes Logistic Regression

In [ ]:
def get_dense_variational_layer(prior_fn, posterior_fn, data):
    dense_variational_layer = tfpl.DenseVariational(
        input_shape=(data.shape[1],),
        units=2,
        make_prior_fn=prior_fn,
        make_posterior_fn=posterior_fn,
        kl_weight=1 / data.shape[0],
        kl_use_exact=False,
    )
    return dense_variational_layer

#### The spike-and-slab prior
We will use a so-called "spike and slab" (also called a *scale mixture prior*) distribution as the prior distribution for the model weights. This distribution has a density that is the weighted sum of two normally distributed ones: one with a standard deviation of 1 and one with a standard deviation of 10. In this way, it has a sharp spike around 0 (from the normal distribution with standard deviation 1), but is also more spread out towards far away values (from the contribution from the normal distribution with standard deviation 10). The reason for using such a prior is that it is like a standard unit normal, but makes values far away from 0 more likely, allowing the model to explore a larger weight space.

In [ ]:
# Function to define the spike and slab distribution
def spike_and_slab(event_shape, dtype):
    distribution = tfd.Mixture(
        cat=tfd.Categorical(probs=[0.5, 0.5]),
        components=[
            tfd.Independent(tfd.Normal(
                loc=tf.zeros(event_shape, dtype=dtype), 
                scale=1.0*tf.ones(event_shape, dtype=dtype)),
                            reinterpreted_batch_ndims=1),
            tfd.Independent(tfd.Normal(
                loc=tf.zeros(event_shape, dtype=dtype), 
                scale=10.0*tf.ones(event_shape, dtype=dtype)),
                            reinterpreted_batch_ndims=1)],
    name='spike_and_slab')
    return distribution

Below we plot what the spike-and-slab distribution looks like.

In [ ]:
x_plot = np.linspace(-5, 5, 1000)[:, np.newaxis]
plt.plot(x_plot, tfd.Normal(loc=0, scale=1).prob(x_plot).numpy(), label='unit normal', linestyle='--')
plt.plot(x_plot, spike_and_slab(1, dtype=tf.float32).prob(x_plot).numpy(), label='spike and slab')
plt.xlabel('x')
plt.ylabel('Density')
plt.legend()
plt.show()

In [ ]:
def get_prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = tf.keras.Sequential([
        tfpl.DistributionLambda(lambda t: spike_and_slab(n, dtype=dtype))
    ])
    return prior_model

In [ ]:
def get_posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = tf.keras.Sequential([
        tfpl.VariableLayer(tfp.layers.IndependentNormal.params_size(n), dtype=dtype),
        tfp.layers.IndependentNormal(n),
    ])
    return posterior_model

In [ ]:
def nll(y_true, y_pred):
    return -y_pred.log_prob(y_true)

In [ ]:
dense_variational_layer = get_dense_variational_layer(get_prior, get_posterior, oh_leaf_nodes)

bayesian_fake_hotel_model = Sequential([
    dense_variational_layer,
    tfpl.OneHotCategorical(2, convert_to_tensor_fn=tfd.Distribution.mode)
])

In [ ]:
bayesian_fake_hotel_model.compile(loss=nll,
              optimizer=Adam(),
              metrics=['accuracy'],
              experimental_run_tf_function=False)

In [ ]:
bayesian_fake_hotel_model.summary()

In [ ]:
y = batman_data_daterange("2020-04-01", "2020-09-01")["is_fake_hotel"].to_numpy()
y = y.reshape((y.shape[0], 1))
gt_oh_enc = OneHotEncoder(
    sparse=False
)
y = gt_oh_enc.fit_transform(y)

In [ ]:
bayesian_fake_hotel_model.fit(x=oh_leaf_nodes, y=y, epochs=10, verbose=True)

In [ ]:
def analyse_model_prediction(data, true_labels, model, instance_idx, run_ensemble=False):
    num_classes = 2
    if run_ensemble:
        ensemble_size = 200
    else:
        ensemble_size = 1
    image = data[instance_idx]
    true_label = int(true_labels[instance_idx, 1])
    predicted_probabilities = np.empty(shape=(ensemble_size, num_classes))
    for i in range(ensemble_size):
        predicted_probabilities[i] = model(image[np.newaxis, :]).mean().numpy()[0]
    model_prediction = model(image[np.newaxis, :])
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(10, 2),
                                   gridspec_kw={'width_ratios': [2, 4]})
    
    # Show the label
    ax1.axis('off')
    ax1.set_title('True label: {}'.format(str(true_label)))
    
    # Show a 95% prediction interval of model predicted probabilities
    pct_2p5 = np.array([np.percentile(predicted_probabilities[:, i], 2.5) for i in range(num_classes)])
    pct_97p5 = np.array([np.percentile(predicted_probabilities[:, i], 97.5) for i in range(num_classes)])    
    bar = ax2.bar(np.arange(num_classes), pct_97p5, color='red')
    bar[int(true_label)].set_color('green')
    ax2.bar(np.arange(num_classes), pct_2p5-0.02, color='white', linewidth=1, edgecolor='white')
    ax2.set_xticks(np.arange(num_classes))
    ax2.set_ylim([0, 1])
    ax2.set_ylabel('Probability')
    ax2.set_title('Model estimated probabilities')
    plt.show()

In [ ]:
def prediction_interval(model, data, instance_idx, num_samples=200):
    """
    This method returns the mean prediction and the width of the 95% prediction interval
    """
    num_classes = 2
    image = data[instance_idx]    
    predicted_probabilities = np.empty(shape=(num_samples, num_classes))
    for i in range(num_samples):
        predicted_probabilities[i] = model(image[np.newaxis, :]).mean().numpy()[0]

    pct_2p5 = np.percentile(predicted_probabilities[:, 1], 2.5)
    pct_97p5 = np.percentile(predicted_probabilities[:, 1], 97.5)
    interval_width = pct_97p5 - pct_2p5
    pred_mean = np.mean(predicted_probabilities[:, 1])
    return pred_mean, interval_width

In [ ]:
for i in range(10):
    analyse_model_prediction(oh_leaf_nodes, y, bayesian_fake_hotel_model, i, run_ensemble=True)

### Computation time to construct the 95% prediction interval

Clearly, the CI-width is instance-dependent, which looks nice. But it also took a time. Let's time how quickly we can make our predictions. We now use the prediction_interval-method, to make sure that we're really measuring the time that it takes to construct the 95%-prediction CI and not measuring the time that is taking to plot (as opposed to analyse_model_prediction).

#### How does the prediction time scale in the number of samples-per-prediction?

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i, num_samples=1) for i in range(10)]

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i, num_samples=10) for i in range(10)]

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i, num_samples=50) for i in range(10)]

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i, num_samples=100) for i in range(10)]

As we would expect, the predictions scale linearly in the number samples-per-prediction. Taking 200 samples per prediction took 35.9 seconds to make 10 predictions, and this took 18.1 seconds when taking 100 samples per prediction. This yields 0.01795 seconds-per-predictions-per-sample.

#### How does the prediction time scale in the number of instances?

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i) for i in range(10)]

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i) for i in range(5)]

In [ ]:
%timeit [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, i) for i in range(1)]

As we would expect, the predictions also scale linearly in the number of predictions. Making 10 predictions takes 36 seconds, resulting in 3.6 seconds per prediction.

#### What if we would vectorize this?
Each time that we make one call to the Bayesian model, the model weights get sampled from the model's posterior distribution over its weights. This means that if we predict for different instances in the same model-call, these predictions will get made using the exact same model weights. This is no problem. 

But when we want to sample predictions for a single instance from the posterior predictive distribution, we need the Bayesian model weights to be sampled again from the model's posterior distribution over its weights. Otherwise, if we would re-use the same Bayesian model weights to generate another prediction on the same instance, that prediction would end up being the same again and we would effectively end up re-generating the same sample from the posterior predictive distribution instead of generating a new one. Thus, multiple predictions for the same instance need to be generated through independent calls to the model if we want these predictions to be different from eachother.

For this reason, we are able to vectorize over the instance-dimension, but we are not able to vectorize over the posterior-predictive-sample-dimension. The latter dimension we will need to keep doing with a for-loop.

In [ ]:
def prediction_interval_vectorized(model, data, instance_idx, num_samples=200):
    """
    This method returns the mean prediction and the width of the 95% prediction interval.
    In contrast to **prediction_interval**, this implementation vectorizes the compute over the instance
    dimension. Pass an array of instance_idx, and the output will be the and array of prediction means and 
    prediction interval widths for those instances.
    """
    num_instances = len(instance_idx)
    image = data[instance_idx]
    predicted_probabilities = np.empty(shape=(num_samples, num_instances))
    for i in range(num_samples):
        predicted_probabilities[i,:] = model(image).mean().numpy()[:,1]
    
    pct_2p5 = np.array([np.percentile(predicted_probabilities[:, i], 2.5) for i in range(num_instances)])
    pct_97p5 = np.array([np.percentile(predicted_probabilities[:, i], 95.5) for i in range(num_instances)])
    interval_width = pct_97p5 - pct_2p5
    pred_mean = np.array([np.mean(predicted_probabilities[:, i]) for i in range(num_instances)])
    return pred_mean, interval_width

In [ ]:
%timeit prediction_interval_vectorized(bayesian_fake_hotel_model, oh_leaf_nodes, range(1))

In [ ]:
%timeit prediction_interval_vectorized(bayesian_fake_hotel_model, oh_leaf_nodes, range(5))

In [ ]:
%timeit prediction_interval_vectorized(bayesian_fake_hotel_model, oh_leaf_nodes, range(10))

#### How many samples-per-prediction do we need?

In the first 6 predictions there are 2 with reasonably large spread: instance 3 and 5 (0-indexed). Let's see how the spread of those two predictions is affected by sampling more (or less).

In [ ]:
def plot_ci_width_for_instance(instance_idx):
    sample_sizes = [2 ** i for i in range(10)]
    ci_width = [
        [prediction_interval(bayesian_fake_hotel_model, oh_leaf_nodes, instance_idx, num_samples=size)[1] for i in range(10)]
        for size in sample_sizes
    ]
    ci_width_pd = pd.DataFrame(
        {"number_of_samples": [sample_sizes[i] for i in range(len(sample_sizes)) for _ in range(10)],
         "ci_width": [item for sublist in ci_width for item in sublist],
        })
    sns.lineplot(
        data=ci_width_pd,
        x="number_of_samples",
        y="ci_width",
    )

In [ ]:
plot_ci_width_for_instance(3)

In [ ]:
sample_sizes = [2 ** i for i in range(10)]
ci_width = np.concatenate([
    prediction_interval_vectorized(bayesian_fake_hotel_model, oh_leaf_nodes, [3], num_samples=size)[1] for i in range(10)
    for size in sample_sizes
], axis=None)

In [ ]:
ci_width

In [ ]:
def plot_ci_width_for_instance_vectorized(instance_idx):
    sample_sizes = [2 ** i for i in range(10)]
    ci_width = np.concatenate([
        prediction_interval_vectorized(bayesian_fake_hotel_model, oh_leaf_nodes, instance_idx, num_samples=size)[1] for i in range(10)
        for size in sample_sizes
    ], axis=None)
    ci_width_pd = pd.DataFrame(
        {"number_of_samples": [sample_sizes[i] for i in range(len(sample_sizes)) for _ in range(10)],
         "ci_width": ci_width,
        })
    sns.lineplot(
        data=ci_width_pd,
        x="number_of_samples",
        y="ci_width",
    )

In [ ]:
plot_ci_width_for_instance_vectorized([3])

#### Uncertainty quantification using entropy

We can also calculate some aggregated statistic that captures the uncertainty of a models predictions across a dataset of several predictions, instead of for individual predictions. One way to do this is to calculate the [entropy](https://en.wikipedia.org/wiki/Entropy_%28information_theory%29) of the distribution. The entropy is the expected information (or informally, the expected 'surprise') of a random variable, and is a measure of the uncertainty of the random variable. The entropy of the estimated probabilities for sample $i$ is defined as

$$
H_i = -\sum_{j=1}^{2} p_{ij} \text{log}_{2}(p_{ij})
$$

where $p_{ij}$ is the probability that the model assigns to sample $i$ corresponding to label $j$. The entropy as above is measured in _bits_. If the natural logarithm is used instead, the entropy is measured in _nats_.

In [ ]:
def get_correct_indices(model, x, labels):
    y_model = model(x)
    correct = np.argmax(y_model.mean(), axis=1) == np.squeeze(labels)
    correct_indices = [i for i in range(x.shape[0]) if correct[i]]
    incorrect_indices = [i for i in range(x.shape[0]) if not correct[i]]
    return correct_indices, incorrect_indices


def plot_entropy_distribution(model, x, labels):
    probs = model(x).mean().numpy()
    entropy = -np.sum(probs * np.log2(probs), axis=1)
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    for i, category in zip(range(2), ['Correct', 'Incorrect']):
        entropy_category = entropy[get_correct_indices(model, x, labels)[i]]
        mean_entropy = np.mean(entropy_category)
        num_samples = entropy_category.shape[0]
        title = category + 'ly labelled ({:.1f}% of total)'.format(num_samples / x.shape[0] * 100)
        axes[i].hist(entropy_category, weights=(1/num_samples)*np.ones(num_samples))
        axes[i].annotate('Mean: {:.3f} bits'.format(mean_entropy), (0.4, 0.9), ha='center')
        axes[i].set_xlabel('Entropy (bits)')
        axes[i].set_ylim([0, 1])
        axes[i].set_ylabel('Probability')
        axes[i].set_title(title)
    plt.show()

In [ ]:
def plot_entropy_distribution(model, x, labels):
    probs = model(x).mean().numpy()
    entropy = -np.sum(probs * np.log2(probs), axis=1)
    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    for i, category in zip(range(2), ['Correct', 'Incorrect']):
        entropy_category = entropy[get_correct_indices(model, x, labels)[i]]
        mean_entropy = np.mean(entropy_category)
        num_samples = entropy_category.shape[0]
        title = category + 'ly labelled ({:.1f}% of total)'.format(num_samples / x.shape[0] * 100)
        axes[i].hist(entropy_category, weights=(1/num_samples)*np.ones(num_samples))
        axes[i].annotate('Mean: {:.3f} bits'.format(mean_entropy), (0.4, 0.9), ha='center')
        axes[i].set_xlabel('Entropy (bits)')
        axes[i].set_ylim([0, 1])
        axes[i].set_ylabel('Probability')
        axes[i].set_title(title)
    plt.show()

In [ ]:
def plot_ci_width_distribution(model, x, labels):
    preds_with_cis = prediction_interval_vectorized(
        model, x, instance_idx=range(len(x))
    )
    ci_widths = preds_with_cis[1]

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    for i, category in zip(range(2), ['Correct', 'Incorrect']):
        ci_widths_category = ci_widths[get_correct_indices(model, x, labels)[i]]
        mean_ci_width = np.mean(ci_widths_category)
        num_samples = ci_widths_category.shape[0]
        title = category + 'ly labelled ({:.1f}% of total)'.format(num_samples / x.shape[0] * 100)
        axes[i].hist(ci_widths_category, weights=(1/num_samples)*np.ones(num_samples))
        axes[i].annotate('Mean 95%-CI-width: {:.3f}'.format(mean_ci_width), (0.4, 0.9), ha='center')
        axes[i].set_xlabel('95%-CI-width')
        axes[i].set_ylim([0, 1])
        axes[i].set_ylabel('Probability')
        axes[i].set_title(title)       
    plt.show()

In [ ]:
plot_entropy_distribution(bayesian_fake_hotel_model, oh_leaf_nodes, [elem[1] for elem in y])

The plot above shows that the model was much more uncertain about its incorrect predictions than about its correct predictions.

In [ ]:
def evaluate_on_timerange(
    start_date,
    end_date,
    measure
):
    # create one-hot-encoded leaf-node-assignments
    leaf_node_assignments = model.predict(batman_data_daterange(start_date, end_date)[feature_cols], pred_leaf=True)
    one_hot_leaf_nodes = oh_enc.transform(leaf_node_assignments)

    # labels
    y = batman_data_daterange(start_date, end_date)["is_fake_hotel"].to_numpy()
    y_one_hot = y.reshape((-1, 1))
    y_one_hot = gt_oh_enc.transform(y_one_hot)

    if measure == "entropy":
        plot_entropy_distribution(bayesian_fake_hotel_model, one_hot_leaf_nodes, y)
    elif measure == "ci_width":
        plot_ci_width_distribution(bayesian_fake_hotel_model, one_hot_leaf_nodes, y)
    else:
        raise ValueError(f"Evaluation measure {measure} is unknown.")

We now evaluate the entropy on the month just after training the model.

In [ ]:
evaluate_on_timerange("2020-09-01", "2020-10-01", measure="entropy")

And for comparison, we now evaluate the entropy on the data from 3 months later.

In [ ]:
evaluate_on_timerange("2020-12-01", "2021-01-01", measure="entropy")

In [ ]:
evaluate_on_timerange("2021-01-01", "2021-02-01", measure="entropy")

In [ ]:
evaluate_on_timerange("2021-02-01", "2021-03-01", measure="entropy")

Notice how over time the entropy of the predictions increased a little bit: 

- from 0.596 to 0.639 for the incorrect predictions
- from 0.286 to 0.314 for the correct predictions

Remember that entropy is a measure of **how wide** the uncertainty intervals around the prediction are. This shows that the model that was trained on the 2020-04-01 to 2020-09-01 date range is less certain about its predictions four months after training compared to its predictions in the month immediately after training.

In [ ]:
evaluate_on_timerange("2020-09-01", "2020-10-01", measure="ci_width")

In [ ]:
evaluate_on_timerange("2020-10-01", "2020-11-01", measure="ci_width")

In [ ]:
evaluate_on_timerange("2020-11-01", "2020-12-01", measure="ci_width")

In [ ]:
evaluate_on_timerange("2020-12-01", "2021-01-01", measure="ci_width")

In [ ]:
evaluate_on_timerange("2021-01-01", "2021-02-01", measure="ci_width")

In [ ]:
evaluate_on_timerange("2021-02-01", "2021-03-01", measure="ci_width")

## Some more plotting

In [ ]:
# create one-hot-encoded leaf-node-assignments
startdate = "2021-02-01"
enddate = "2021-03-01"
leaf_node_assignments = model.predict(batman_data_daterange(startdate, enddate)[feature_cols], pred_leaf=True)
one_hot_leaf_nodes = oh_enc.transform(leaf_node_assignments)

# labels
y = batman_data_daterange(startdate, enddate)["is_fake_hotel"].to_numpy()
y_one_hot = y.reshape((-1, 1))
y_one_hot = gt_oh_enc.transform(y_one_hot)
y_hat, ci_widths = prediction_interval_vectorized(bayesian_fake_hotel_model, 
                                                  one_hot_leaf_nodes, 
                                                  instance_idx=range(len(one_hot_leaf_nodes)))

In [ ]:
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 18

ax = sns.scatterplot(x=ci_widths, 
                y=y_hat, 
                hue=y, 
                alpha=0.25)
ax.set(xlabel='Width of the 95%-interval of the posterior predictive distribution', 
       ylabel='The MLE of P(y="fake hotel" | X)')
ax.legend().set_title("Is fake hotel")

In [ ]:
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 18

ax = sns.scatterplot(x=np.log10(ci_widths)[y==0], 
                y=np.log10(y_hat)[y==0],  
                alpha=0.25)
ax = sns.scatterplot(x=np.log10(ci_widths)[y==1], 
                y=np.log10(y_hat)[y==1],
                alpha=0.5,
                ax=ax
                )
ax.set(xlabel='Log - Width of the 95%-interval of the posterior predictive distribution', 
       ylabel='Log - The MLE of P(y="fake hotel" | X)')
ax.legend().set_title("Is fake hotel")

In [ ]:
def plot_ci_width_distribution(model, x, labels):
    preds_with_cis = prediction_interval_vectorized(
        model, x, instance_idx=range(len(x))
    )
    ci_widths = preds_with_cis[1]

    fig, axes = plt.subplots(1, 2, figsize=(10, 4))
    for i, category in zip(range(2), ['Correct', 'Incorrect']):
        ci_widths_category = ci_widths[get_correct_indices(model, x, labels)[i]]
        mean_ci_width = np.mean(ci_widths_category)
        num_samples = ci_widths_category.shape[0]
        title = category + 'ly labelled ({:.1f}% of total)'.format(num_samples / x.shape[0] * 100)
        axes[i].hist(ci_widths_category, weights=(1/num_samples)*np.ones(num_samples))
        axes[i].annotate('Mean 95%-CI-width: {:.3f}'.format(mean_ci_width), (0.4, 0.9), ha='center')
        axes[i].set_xlabel('95%-CI-width')
        axes[i].set_ylim([0, 1])
        axes[i].set_ylabel('Probability')
        axes[i].set_title(title)       
    plt.show()

#### January

In [ ]:
# create one-hot-encoded leaf-node-assignments
startdate = "2021-01-01"
enddate = "2021-02-01"
leaf_node_assignments = model.predict(batman_data_daterange(startdate, enddate)[feature_cols], pred_leaf=True)
one_hot_leaf_nodes = oh_enc.transform(leaf_node_assignments)

# labels
y = batman_data_daterange(startdate, enddate)["is_fake_hotel"].to_numpy()
y_one_hot = y.reshape((-1, 1))
y_one_hot = gt_oh_enc.transform(y_one_hot)
y_hat, ci_widths = prediction_interval_vectorized(bayesian_fake_hotel_model, 
                                                  one_hot_leaf_nodes, 
                                                  instance_idx=range(len(one_hot_leaf_nodes)))

In [ ]:
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 18

ax = sns.scatterplot(x=ci_widths, 
                y=y_hat, 
                hue=y, 
                alpha=0.25)
ax.set(xlabel='Width of the 95%-interval of the posterior predictive distribution', 
       ylabel='The MLE of P(y="fake hotel" | X)')
ax.legend().set_title("Is fake hotel")

In [ ]:
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 18

ax = sns.scatterplot(x=np.log10(ci_widths)[y==0], 
                y=np.log10(y_hat)[y==0],  
                alpha=0.25)
ax = sns.scatterplot(x=np.log10(ci_widths)[y==1], 
                y=np.log10(y_hat)[y==1],
                alpha=0.5,
                ax=ax
                )
ax.set(xlabel='Log - Width of the 95%-interval of the posterior predictive distribution', 
       ylabel='Log - The MLE of P(y="fake hotel" | X)')
ax.legend().set_title("Is fake hotel")

#### December

In [ ]:
# create one-hot-encoded leaf-node-assignments
startdate = "2020-12-01"
enddate = "2021-01-01"
leaf_node_assignments = model.predict(batman_data_daterange(startdate, enddate)[feature_cols], pred_leaf=True)
one_hot_leaf_nodes = oh_enc.transform(leaf_node_assignments)

# labels
y = batman_data_daterange(startdate, enddate)["is_fake_hotel"].to_numpy()
y_one_hot = y.reshape((-1, 1))
y_one_hot = gt_oh_enc.transform(y_one_hot)
y_hat, ci_widths = prediction_interval_vectorized(bayesian_fake_hotel_model, 
                                                  one_hot_leaf_nodes, 
                                                  instance_idx=range(len(one_hot_leaf_nodes)))

In [ ]:
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 18

ax = sns.scatterplot(x=ci_widths, 
                y=y_hat, 
                hue=y, 
                alpha=0.25)
ax.set(xlabel='Width of the 95%-interval of the posterior predictive distribution', 
       ylabel='The MLE of P(y="fake hotel" | X)')
ax.legend().set_title("Is fake hotel")

In [ ]:
plt.figure(figsize=(10, 10))
plt.rcParams["font.size"] = 18

ax = sns.scatterplot(x=np.log10(ci_widths)[y==0], 
                y=np.log10(y_hat)[y==0],  
                alpha=0.25)
ax = sns.scatterplot(x=np.log10(ci_widths)[y==1], 
                y=np.log10(y_hat)[y==1],
                alpha=0.5,
                ax=ax
                )
ax.set(xlabel='Log - Width of the 95%-interval of the posterior predictive distribution', 
       ylabel='Log - The MLE of P(y="fake hotel" | X)')
ax.legend().set_title("Is fake hotel")

### Conclusion
This is merely a Proof-of-Concept of a Variational Bayesian model with uncertainty quantification of its predictions. There is still much more validation needed, and there are still many follow-up questions, such as:

- are the data points with wide prediction intervals indeed the data points that are useful for manual review (in an active learning setup)
- does the pattern of increasing uncertainty over time hold more generally, beyond the example shown above?
- can an increase in prediction uncertainty be useful detection of model degradation as a result of concept drift already before we have our labels, or it is too unreliable for this use case?
- how sensitive is the Variational model to number of trees in the tree ensemble?

And many more. Yet, this notebook offers a first exploration and proof-of-concept of the topic and was as much as I managed to do in the timeframe of a single hackathon. I hope that in the adaptive ML task force we can continue to explore several of the questions that I formulated above, and that this notebook sparks some early feedback.

In [ ]:
2